# init
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de

## Tasks
- get OpenStreetMap data using the osmnx functionalities in the Overpass API
- clip it to an AOI

## Prerequisites
- have osmnx and geopandas installed
- have an AOI saved as a vector (shapefile or geopackage)

In [ ]:
import osmnx as ox
import geopandas as gpd
from pathlib import Path

# imports

In [ ]:
#import the AOI
AOI_path = Path('../data/processed/')
AOI_file = 'JAK_LIM_BoundingBox_AOI_A.shp'
AOI_gdf = gpd.read_file(AOI_path / AOI_file)

In [ ]:
#bounds provides minx, miny, maxx, maxy
aoi_coords = AOI_gdf.to_crs(epsg='4326').envelope
bbox_df = gpd.GeoDataFrame(geometry=aoi_coords)
print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])

In [ ]:
#get waterway linear  features from OSM
# List key-value pairs for tags
tags = {'waterway': ['river', 'stream', 'tidal_channel', 'canal', 'drain', 'ditch', 
                     'pressurised','fairway','fish_pass','canoe_pass']} # see full list of classes at https://wiki.openstreetmap.org/wiki/Waterways
col_list = ['osmid','waterway','intermittent','usage','geometry'] #OSM has hundreds of cols, we just want a few. 

water_linear = ox.features_from_bbox(north=aoi_coords.bounds.values[0][3],
                              south=aoi_coords.bounds.values[0][1],
                              west=aoi_coords.bounds.values[0][0],
                              east=aoi_coords.bounds.values[0][2],
                              tags=tags)
water_linear.reset_index(inplace=True)

#filtering out the wrong geometries (POLYGON or POINT) and element types (NODE)
water_linear = water_linear.loc[(water_linear.geometry.type=='LineString'), col_list] #&(water_linear.element_type == 'way')
#convert the CRS of roads to the same as AOI
water_linear.to_crs(AOI_gdf.crs,inplace=True)

#plot them
water_linear.plot('waterway',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', linewidth=1, zorder=2)

In [ ]:
#clip using geopandas, the mask is the AOI we imported above
water_clipped_gdf = gpd.clip(water_linear, mask=AOI_gdf, keep_geom_type=True)
water_clipped_gdf.plot('waterway',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', linewidth=1, zorder=2)

In [ ]:
#get waterway area features from OSM
# List key-value pairs for tags
tags = {'water': ['lagoon','lake','oxbow','rapids','river','stream','stram_pool',
                  'basin','canal', 'ditch', 'fish_pass','harbour', 'lock', 'moat', 
                  'pond', 'reflecting_pool', 'reservoir', 'wastewater']} # see full list of classes at https://wiki.openstreetmap.org/wiki/Key:water
col_list = ['osmid','water','natural','source','geometry','name'] #OSM has hundreds of cols, we just want a few. 

water_area = ox.features_from_bbox(north=aoi_coords.bounds.values[0][3],
                              south=aoi_coords.bounds.values[0][1],
                              west=aoi_coords.bounds.values[0][0],
                              east=aoi_coords.bounds.values[0][2],
                              tags=tags)
water_area.reset_index(inplace=True)

#filtering out the wrong geometries (POLYGON or POINT) and element types (NODE)
water_area = water_area.loc[(water_area.geometry.type=='Polygon'), col_list] #&(water_linear.element_type == 'way')
#convert the CRS of roads to the same as AOI
water_area.to_crs(AOI_gdf.crs,inplace=True)

#plot them
water_area.plot('water',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', linewidth=1, zorder=2)

In [ ]:
#get waterway area features from OSM
# List key-value pairs for tags
tags = {'natural': ['water','coastline']} # see full list of classes at https://wiki.openstreetmap.org/wiki/Key:water
col_list = ['osmid','water','natural','source','geometry','name'] #OSM has hundreds of cols, we just want a few. 

water_area = ox.features_from_bbox(north=aoi_coords.bounds.values[0][3],
                              south=aoi_coords.bounds.values[0][1],
                              west=aoi_coords.bounds.values[0][0],
                              east=aoi_coords.bounds.values[0][2],
                              tags=tags)
water_area.reset_index(inplace=True)

#filtering out the wrong geometries (POLYGON or POINT) and element types (NODE)
water_area = water_area.loc[(water_area.geometry.type=='Polygon'), col_list] #&(water_linear.element_type == 'way')
#convert the CRS of roads to the same as AOI
water_area.to_crs(AOI_gdf.crs,inplace=True)

#plot them
water_area.plot('water',ax=AOI_gdf.plot(figsize=(10, 10),  alpha=0.5, color='lightgray'),cmap='Paired', linewidth=1, zorder=2)

# export

In [ ]:
#export roads  to a shapefile
water_clipped_gdf.to_file(AOI_path / 'JAK_HYD_waterways_linear_OSM_2024_L.shp')

In [ ]:
#export places to a shapefile
water_area.to_file(AOI_path / 'JAK_HYD_waterways_area_OSM_2024_A.shp')